In [3]:
from dataloader import Dataloader
data = Dataloader()
# Aproach 1 only listing data 
# Aproach 2 only reviews (Advanced analysis)
# Aproach 3 comninede

# Develop clustering o present vocabulary to express the different dimension (clean, communication checkin, value)
# Use bucket of 0-1 1-2 2-3 3-4 4-5
# 1 model for each dimension 

# Test more cities

In [19]:
conlumns = list(filter(lambda c: 'review' in c , data.getListings().columns))
data.getListings().sample(3)[conlumns]
data.getListings().sample(3)

AttributeError: 'dict' object has no attribute 'getListings'

In [ ]:
data.getComments().sample(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
267846,32485135,608997347,2020-02-22,80334626,Michael,CityHub was amazing! Everything from tips to l...
151290,11404636,595987086,2020-01-21,182982478,Frances,Great place! Lovely host! Great location!
280260,36609293,607018365,2020-02-17,17791112,Diana,The location it’s amazing! So beautiful view f...


In [2]:
# Comments from listing
listings = data.getListings()[data.getListings()['id'] == 32707589]
filtered_comments = data.getCommentsFromListing(listings)  # or data.getCommentsFromListing(32707589) 
filtered_comments.sample(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
271688,32707589,578581887,2019-12-18,25809087,Courtenay,"Christiaan & Onno were wonderful hosts, I happ..."
271722,32707589,615127297,2020-03-07,55959365,Nancypytlak@Yahoo.Com,"Will not be disappointed, apartment was real c..."
271736,32707589,604813076749525664,2022-04-14,237691117,Marjorie,Wonderful


In [ ]:
# more than 1 listing
listings = data.getListings().sample(3)
filtered_comments = data.getCommentsFromListing(listings) 
filtered_comments.sample(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
268017,31050856,701920689227623604,2022-08-26,1885345,Marijka,A wonderful place to stay. As a family of fou...
163307,13389882,282710286,2018-06-28,95537408,Jade,Great apartment in a really lovely part of Ams...
199525,18576284,170710470,2017-07-16,33794387,Sergi,Joanneke's place is ideal for a first visit to...


In [ ]:
# Single comments 
comments = data.getComments()[data.getComments()['id'] == 466296625]['listing_id']

filtered_listing =  data.getListings()[data.getListings()['id'].isin(comments) ]
#filtered_listing = data.getListingByComments(comments) # or data.getListingByComments(466296625) 
filtered_listing

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
3159,32707589,https://www.airbnb.com/rooms/32707589,20221205143541,2022-12-05,city scrape,Spacious 4p apt at the best location you can get!,Great spacious apartment on the most famous ca...,NaN,https://a0.muscache.com/pictures/7798ffc5-adfa...,245938143,...,4.94,4.95,4.62,0363 6EF3 971C B15C C2FC,t,1,0,1,0,1.47


In [ ]:
# More than 1 comment 
comments = data.getComments().sample(3)
filtered_listing = data.getListingByComments(comments) # or data.getListingByComments([id1, id2]) 
filtered_listing.sample(2)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
976,8749341,https://www.airbnb.com/rooms/8749341,20221205143541,2022-12-05,city scrape,Centre - Rooftop Terrace & Unique Colorful Room,Central location. Large private to guests roof...,The house is in the centre of Amsterdam one st...,https://a0.muscache.com/pictures/f72730f0-c5eb...,45891368,...,4.96,4.99,4.79,0363 B897 E97C E11D 00DE,f,1,0,1,0,3.80
2387,23177757,https://www.airbnb.com/rooms/23177757,20221205143541,2022-12-05,city scrape,Pretty studio on a ship in the Heart of Amsterdam,"The Morgenster is a iron ship, build in 1905. ...",The ship is located at a ten minutes walk from...,https://a0.muscache.com/pictures/fd3c241e-f8c8...,7671673,...,4.98,4.91,4.61,0363 48F4 0942 0247 54BF,f,1,0,1,0,2.87


In [41]:
from amenities import extract_amenities
import json
import itertools
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
#test = extract_amenities(listings)
listings = data.getListings()
amenities = listings[['id', 'amenities']]
amenities['amenities'] = amenities['amenities'].apply(lambda x: json.loads(x))
amenities_uniques = set(sorted(list(itertools.chain.from_iterable(amenities['amenities'].values))))
amenities_map = np.zeros( (amenities.shape[0], len(amenities_uniques)) )
for col, row_amenities in enumerate(amenities_uniques):
    for row, listing_amenities in amenities['amenities'].items():
        if row_amenities in listing_amenities: amenities_map[row][col] == 1 

c:\Users\matti\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Game console: Nintendo Wii and PS4
Brandless body soap
40" HDTV with standard cable, Netflix, premium cable, Chromecast
70" HDTV with Disney+, Netflix, premium cable
Harman / kardon Bluetooth sound system
Heating - split type ductless system
Radiant heating
Fast wifi – 164 Mbps
Marshall  sound system
JBL Xtreme Bluetooth sound system
Fast wifi – 246 Mbps
Syos conditioner
Stainless steel oven
TV with standard cable, Chromecast
Ikea oven
Verschillend conditioner
55" HDTV with Amazon Prime Video, Netflix, standard cable, Apple TV, premium cable, HBO Max
Ikea refrigerator
35" HDTV with Disney+, Netflix
Care body soap
Eco-friendly body soap
Paid parking lot on premises
HDTV with Apple TV, Netflix, premium cable, standard cable
ATAG gas stove
Fast wifi – 97 Mbps
Coffee maker: drip coffee maker, french press
Falcon oven
Green soap shampoo
Game console: PS5
62" HDTV with Netflix, Disney+, HBO Max, Amazon Prime Video
HDTV with Fire TV, DVD player, Netflix
L oreal conditioner
Self check-in
43" H

In [40]:
#amenities['amenities']
pd.DataFrame(columns=amenities_uniques,data=amenities_map)['Brandless body soap']


0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6804    0.0
6805    0.0
6806    0.0
6807    0.0
6808    0.0
Name: Brandless body soap, Length: 6809, dtype: float64